In [1]:
asdf = {'isAccident':True, 'isObstacle':False}

In [20]:
import psycopg2 as ps
from wordcloud import WordCloud
import  matplotlib.pyplot as plt 
import re

conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
option = 'day'
cursor = conn.cursor()
q = '''SELECT TWEET_TEXT AS TEXT 
        FROM TWEET
        WHERE TWEET_CREATED>(
            SELECT DATE_TRUNC('DAY', MAX(TWEET_CREATED) - INTERVAL '%i %s') 
            FROM TWEET
        ); ''' % (
            (7 if option=='week' else 1), 
            'day' if option=='week' else option)
cursor.execute(q)
results = cursor.fetchall()
cursor.close()
conn.close()

tweets = ' '.join([Preprocessing.preprocess(t[0]) for t in results])
# tweets = ' '.join(re.findall('([A-Z]\S+)', tweets))

In [15]:
import sys
sys.path.append('./TweetData')
from TweetData import keys, Preprocessing

In [ ]:
wc = WordCloud(background_color='white', 
    width=350, height=310).generate(tweets)

plt.figure(figsize=(15,9))
plt.imshow(wc)
plt.axis('off')
plt.show()

In [1]:
import time 

In [5]:
count = 0
while True:
    time.sleep(1)
    count += 1
    print('1')
    if count==5: False

1
1
1
1
1
1
1


KeyboardInterrupt: 

In [1]:
import pandas as pd

In [15]:
data = pd.DataFrame([[1, 2, 3], [3, 4, 5], [6, 7, 8]], 
    columns=['one', 'two', 'three'])
data

,one,two,three
0,1,2,3
1,3,4,5
2,6,7,8


In [16]:
data2 = pd.DataFrame([[3, 4, 5], [7, 8, 9], [1, 3, 5]], 
    columns=['one', 'two', 'three'])
data2

,one,two,three
0,3,4,5
1,7,8,9
2,1,3,5


In [5]:
string = 'esto es una prueba'
[word for word in string.split(' ')]

['esto', 'es', 'una', 'prueba']

In [2]:
data3 = pd.DataFrame(columns=['one', 'two', 'three'])
len(data3)

0

In [17]:
data.merge(data2, how='outer')

,one,two,three
0,1,2,3
1,3,4,5
2,6,7,8
3,7,8,9
4,1,3,5


In [8]:
for index, i in data.iterrows():
    print(i.one)

1


In [41]:
max(data.iloc[0,:])

3

In [11]:
data.columns.values.tolist()

['one', 'two', 'three']

In [1]:
import psycopg2 as ps
from getpass import getpass
import time, datetime

In [3]:
conn = ps.connect(database='traffictwt', user='postgres', password='root')

option = 'day'
query = '''
    SELECT
        COUNT(CASE WHEN I.ISACCIDENT THEN 1 END) AS ACCIDENTS,
        COUNT(CASE WHEN I.ISOBSTACLE THEN 1 END) AS OBSTACLES,
        COUNT(CASE WHEN I.ISDANGER THEN 1 END) AS DANGERS
    FROM TWTINCIDENT AS I LEFT JOIN TWTTWEET AS T
    ON I.INC_TWEET_ID=T.TWEET_ID
    WHERE T.TWEET_CREATED>(
        SELECT DATE_TRUNC('DAY', MAX(TWEET_CREATED) - INTERVAL '%i' %s) 
        FROM TWTTWEET
    );
    ''' % ((7 if option=='day' else 1), option)

cursor = conn.cursor()
try:
    cursor.execute(query)
    results = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
except Exception as e: print(e)
cursor.close()
results

[(83, 188, 3)]

In [20]:
colnames

['accidents', 'obstacles', 'dangers']

In [19]:
results

(83, 188, 3)

In [5]:
import plotly.graph_objects as go
from TweetData import DBConnect, keys

db = DBConnect.DB_Connection()
# db.connect(password=keys.db_pass)

In [16]:
db.connect(password=keys.db_pass)
q = '''
SELECT
    COUNT(CASE WHEN I.ISACCIDENT THEN 1 END) AS ACCIDENTS,
    COUNT(CASE WHEN I.ISOBSTACLE THEN 1 END) AS OBSTACLES,
    COUNT(CASE WHEN I.ISDANGER THEN 1 END) AS DANGERS
FROM TWTINCIDENT AS I LEFT JOIN TWTTWEET AS T
ON I.INC_TWEET_ID=T.TWEET_ID
WHERE T.TWEET_CREATED>(
    SELECT DATE_TRUNC('DAY', MAX(TWEET_CREATED) - INTERVAL '%i' %s) 
    FROM TWTTWEET
); ''' % ((7 if option=='week' else 1), option)
colnames, results = db.query(q)
db.close_connection()
df = pd.DataFrame(results, columns=colnames)
df

['accidents', 'obstacles', 'dangers'] [(16, 17, 0)]


,accidents,obstacles,dangers
0,16,17,0


In [7]:
q_time = '''SELECT DATE_TRUNC('{0}', TWEET_CREATED) AS D, 
                COUNT(TWEET_CREATED) FROM TWTTWEET 
                GROUP BY D ORDER BY D '''.format(option)
colnames1, results1 = [], []
colnames1, results1 = db.query(q_time)